In [ ]:
# 📊 Evaluación de la Red Neuronal Convolucional (CNN)

# 🔧 Cargar librerías
import os
import sys
sys.path.append("../src")

import yaml
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

import evaluate as ev
import utils

In [ ]:
# 📁 Cargar configuración
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
# 📂 Cargar generador de validación
from tensorflow.keras.preprocessing.image import ImageDataGenerator

val_path = os.path.join(config["paths"]["prepared_data"], "val")
img_height = config["image"]["height"]
img_width = config["image"]["width"]
batch_size = config["training"]["batch_size"]

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
# 💾 Cargar modelo entrenado
model_path = config["paths"]["model"]
model = load_model(model_path)

In [ ]:
# 🔮 Predecir probabilidades
y_scores = model.predict(val_generator)
y_true = val_generator.classes
class_names = list(val_generator.class_indices.keys())

In [ ]:
# 🔢 Convertir predicciones a clases
y_pred = np.argmax(y_scores, axis=1)

In [ ]:
# 📉 Matriz de Confusión multiclase
ev.plot_confusion_matrix(y_true, y_pred, class_names=class_names)

In [ ]:
# 🧪 Métricas de clasificación
from sklearn.metrics import classification_report

# ✅ Mostrar métricas por clase
print("📄 Reporte de clasificación por clase:")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

In [ ]:
# 📈 Distribución de probabilidades para cada clase
for i, class_name in enumerate(class_names):
    ev.plot_prediction_distribution(y_true, y_scores[:, i], class_index=i, class_name=class_name)

In [ ]:
# ✅ Reflexión final
print("✅ Evaluación completa. Analiza si hay overfitting, errores sistemáticos o clases mal representadas.")